# PET Voxel Value to SUV Conversion

This notebook converts the voxel values in preprocessed PET images into Standardized Uptake Values (SUV). This is a critical step for standardizing the PET imaging data, making it comparable across different patients and scans.

The process involves the following steps:
1.  **Load Participant Data**: The `participants.tsv` file is loaded to retrieve the `subject_id` and the corresponding `SUV_ratio` for each patient.
2.  **Iterate Through Subjects**: The script loops through each subject listed in the participants table.
3.  **Apply SUV Conversion**: For each subject, the corresponding preprocessed PET image (`..._pet.nii.gz`) is loaded. Each voxel in the image is then multiplied by the patient-specific `SUV_ratio`.
4.  **Save SUV-scaled Image**: The new image, now in SUV scale, is saved with a `_desc-suv` suffix (`..._pet_desc-suv.nii.gz`) in the same directory. This preserves the original image file.


In [1]:

import pandas as pd
import SimpleITK as sitk
import os
from tqdm.notebook import tqdm

# --- Configuration ---
participants_path = '../../metadata/participants.tsv'
preprocessed_dir = '../../data/1_preprocessed'

# --- Load Data ---
try:
    df_participants = pd.read_csv(participants_path, sep='\t', dtype={'subject_id': str})
    print(f"Successfully loaded {len(df_participants)} records from {participants_path}")
except FileNotFoundError:
    print(f"ERROR: The file {participants_path} was not found. Please ensure the path is correct.")
    # Stop execution if the file is not found
    # This prevents further errors down the line.
    # In a real script, you might raise an exception here.
    df_participants = pd.DataFrame()

# --- Main Processing Loop ---
if not df_participants.empty:
    # Create a dictionary for quick lookup of SUV ratios
    suv_ratio_map = pd.to_numeric(df_participants.set_index('subject_id')['SUV_ratio'], errors='coerce').to_dict()

    # Use tqdm to create a progress bar
    for subject_id, suv_ratio in tqdm(suv_ratio_map.items(), desc="Converting PET to SUV"):
        
        # Check if suv_ratio is valid
        if pd.isna(suv_ratio):
            print(f"Skipping {subject_id}: Invalid or missing SUV_ratio.")
            continue

        # Define file paths
        subject_dir = os.path.join(preprocessed_dir, f"sub-{subject_id}")
        pet_path = os.path.join(subject_dir, f"sub-{subject_id}_pet.nii.gz")
        output_path = os.path.join(subject_dir, f"sub-{subject_id}_pet_desc-suv.nii.gz")

        # Check if the source PET file exists
        if not os.path.exists(pet_path):
            # print(f"Warning: PET file not found for subject {subject_id} at {pet_path}. Skipping.")
            continue
            
        try:
            # Read the PET image
            pet_image = sitk.ReadImage(pet_path, sitk.sitkFloat64)
            
            # Multiply by the SUV ratio
            suv_image = pet_image * suv_ratio
            
            # Save the new image
            sitk.WriteImage(suv_image, output_path)
            
        except Exception as e:
            print(f"ERROR processing {subject_id}: {e}")

    print("\\nConversion process finished.")
else:
    print("Could not proceed with conversion as participant data failed to load.")


Successfully loaded 1061 records from ../../metadata/participants.tsv


Converting PET to SUV:   0%|          | 0/1058 [00:00<?, ?it/s]

\nConversion process finished.


In [2]:

import shutil

# --- Configuration for Copying Files ---
source_subjects_dir = '../../outputs/temp_resample_test'
output_dir = '../../outputs/temp_suv_convert'
preprocessed_base_dir = '../../data/1_preprocessed'

# --- Get Subject IDs from the source directory ---
if os.path.exists(source_subjects_dir):
    subject_sub_dirs = [d for d in os.listdir(source_subjects_dir) if os.path.isdir(os.path.join(source_subjects_dir, d))]
    subject_ids = [d.replace('sub-', '') for d in subject_sub_dirs]
    print(f"Found {len(subject_ids)} subjects to copy: {subject_ids}")

    # --- Create output directory if it doesn't exist ---
    os.makedirs(output_dir, exist_ok=True)

    # --- Copy files for each subject ---
    for subject_id in tqdm(subject_ids, desc="Copying files for inspection"):
        
        # Define subject-specific directories
        subject_output_dir = os.path.join(output_dir, f"sub-{subject_id}")
        os.makedirs(subject_output_dir, exist_ok=True)
        
        source_data_dir = os.path.join(preprocessed_base_dir, f"sub-{subject_id}")

        # --- Define file paths ---
        # Original PET file
        original_pet_path = os.path.join(source_data_dir, f"sub-{subject_id}_pet.nii.gz")
        # SUV converted PET file
        suv_pet_path = os.path.join(source_data_dir, f"sub-{subject_id}_pet_desc-suv.nii.gz")

        # --- Copy files if they exist ---
        # Copy original PET
        if os.path.exists(original_pet_path):
            shutil.copy(original_pet_path, subject_output_dir)
        else:
            print(f"Warning: Original PET file not found for {subject_id} at {original_pet_path}")

        # Copy SUV converted PET
        if os.path.exists(suv_pet_path):
            shutil.copy(suv_pet_path, subject_output_dir)
        else:
            print(f"Warning: SUV PET file not found for {subject_id} at {suv_pet_path}")

    print("\\nFile copying process finished.")
else:
    print(f"ERROR: The source directory {source_subjects_dir} was not found.")


Found 5 subjects to copy: ['AKHRETTICHMONIKA20130930', 'AKHFRHWALDELFRIEDE20190319', 'AKHDOPPELREITERKARL20230123', 'NM042', 'NM247']


Copying files for inspection:   0%|          | 0/5 [00:00<?, ?it/s]

\nFile copying process finished.
